#Process Missing Lyrics
**This Notebook is run in a fairly ad-hoc manner, based on needs. It acts on output from [Data-Exploration Notebook](Data-Exploration.ipynb), namely [missing_lyricsdf.csv](../../data/conditioned/missing_lyricsdf.csv)**

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [51]:
# Additional Config
from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests
import time
import itertools
import json
import pickle

In [5]:
missingdf = pd.DataFrame.from_csv("../../data/conditioned/missing_lyricsdf.csv")

In [6]:
missingdf.head()

,year,title.href,title,artist,lyrics,decade,song_key
position,,,,,,,
2,1970,https://en.wikipedia.org/wiki/(They_Long_to_Be...,(They Long to Be) Close to You,The Carpenters,x,1970,1970-2
7,1970,https://en.wikipedia.org/wiki/I%27ll_Be_There_...,I'll Be There,The Jackson 5,x,1970,1970-7
9,1970,https://en.wikipedia.org/wiki/Let_It_Be_(song),Let It Be,The Beatles,x,1970,1970-9
15,1970,https://en.wikipedia.org/wiki/ABC_(song),ABC,The Jackson 5,x,1970,1970-15
16,1970,https://en.wikipedia.org/wiki/The_Love_You_Save,The Love You Save,The Jackson 5,x,1970,1970-16


## Missing Lyric Alternatives (Went with Lyrics.Wikia)

###MusixMatch Potential

In [10]:
# get lyrics for each song, these are stored individually in harvest folder by song_key
#see https://developer.musixmatch.com/
# e.g. http://api.musixmatch.com/ws/1.1/track.search?apikey=bce08674485ff8c8fbedcdf033107d49&q_artist=queen&q_track=we%20are%20the%20champions&format=json&page_size=1&f_has_lyrics=1

mm_key = "bce08674485ff8c8fbedcdf033107d49"
mm_root = "http://api.musixmatch.com/ws/1.1/"

mm_search_template = "{}track.search?apikey={}&q_artist=%artist&q_track=%track&format=json&page_size=1&f_has_lyrics=1".format(mm_root,mm_key)
mm_lyrics_template = "{}track.lyrics.get?apikey={}&track_id=%track_id".format(mm_root,mm_key)

print "mm_search_template --> ", mm_search_template
print "mm_lyrics_template --> ", mm_lyrics_template

"""
UNFORTUNATELY, Musixmatch only gives 30% lyrics for non-commercial!!!
"""


mm_search_template -->  http://api.musixmatch.com/ws/1.1/track.search?apikey=bce08674485ff8c8fbedcdf033107d49&q_artist=%artist&q_track=%track&format=json&page_size=1&f_has_lyrics=1
mm_lyrics_template -->  http://api.musixmatch.com/ws/1.1/track.lyrics.get?apikey=bce08674485ff8c8fbedcdf033107d49&track_id=%track_id


###SongLyrics is an Alternative if lyrics not in Lyrics.Wikia

In [ ]:
# `song_key`: 1970-1
# from http://www.songlyrics.com/simon-and-garfunkel/bridge-over-troubled-water-lyrics/
"""
<p id="songLyricsDiv"  class="songLyricsV14 iComment-text">Simon And Garfunkel<br />
Bridge Over Troubled Water<br />
Bridge Over Troubled Water<br />
When you're weary, feeling small <br />
When tears are in your eyes, <br />
I will dry them all <br />
I'm on your side <br />
When times get rough <br />
And friends just can't be found <br />
<br />
Like a bridge over troubled water <br />
I will lay me down <br />
Like a bridge over troubled water <br />
I will lay me down <br />
<br />
When you're down and out <br />
When you're on the street <br />
When evening falls so hard <br />
I will comfort you <br />
I'll take your part <br />
When darkness comes <br />
And pain is all around <br /><img src="http://static.urx.io/units/web/urx-unit-loader.gif" style="display:none;" onload="var a=document.createElement('script');a.setAttribute('src','https://static.urx.io/units/web/728893b3-309e-424e-99c8-2213c9ddd1b0.min.js'),this.parentNode.insertBefore(a,this);"><br />
<br />
Like a bridge over troubled water <br />
I will lay me down <br />
Like a bridge over troubled water <br />
I will lay me down <br />
<br />
Sail on silver girl <br />
Sail on by <br />
Your time has come to shine <br />
All your dreams are on their way <br />
See how they shine <br />
When you need a friend <br />
I'm sailing right behind <br />
<br />
Like a bridge over troubled water <br />
I will ease your mind <br />
Like a bridge over troubled water <br />
I will ease your mind<br /></p>
"""

##Process Using Lyrics.Wikia

In [179]:
def cachedRefsOrBuildFromLyricsWikia(song_key, r_cache, url_root, payload):
    #if in the cache just return it
    if (song_key in r_cache):
        print "... using song_key from cache: ", song_key
        return r_cache[song_key],True
    
    # otherwise, attempt to download
    r = requests.get(url_root, params=payload)
    print("... attempting retrieval: ",r.url)
    
    try:
        # here we access the webpage and download the content using requests, just keeping text
        if r.status_code == 200:            
            text = r.text
            r_cache[song_key] = text
            return text,False
        
        # status code not 200
        return None,False 
    except Exception as e:
        print "exception processing song_key --> ", song_key
        print e
        return None,False

def refsFromLyricsWikia(df, seed_cache=None, query_delay=1):
    """
    Attempt to get refs from lyrics.wikia via API, if not valid in cache.
    
    URL e.g. http://lyrics.wikia.com/wiki/Paul_Simon:Bridge_Over_Troubled_Water
    API e.g. http://lyrics.wikia.com/api.php?action=lyrics&artist=Joe%20Bonamassa&song=So%20Many%20Roads&fmt=json
    
     --- Input ---
    df: Dataframe from which to build and cache results
    seed_cache: optional result cache to start from, default None
    query_delay: optional delay value, default 1
    
    --- Return ---
    tuple of the following:
    refs dictionary, list of issues by song_key, cache dictionary (refs + seed_cache)
    """   
    refs = {}
    issues = []
    r_cache = {}
    if seed_cache:
        r_cache = seed_cache
    
    url_root = "http://lyrics.wikia.com/api.php"
    for r in df.iterrows():
        song_key = r[1]['song_key'] 
        artist = r[1]['artist']
        song = r[1]['title']
               
        print "... song_key: {}, artist: {}, song: {}".format(song_key,artist,song)
        payload = {'action': 'lyrics', 'fmt': 'json', 'artist': artist, 'song': song }
        
        t_tuple = cachedRefsOrBuildFromLyricsWikia(song_key, r_cache, url_root, payload)
        text = t_tuple[0]
        is_cache = t_tuple[1]
        
        if text:
            refs[song_key] = text
        else:
            issues.append(song_key)
        
        # only time delay if results had to be downloaded
        if not is_cache:
            time.sleep(query_delay)  # Delay between queries
        
    return refs, issues, r_cache

In [176]:
# quick test #1
lw_t_refs, lw_t_issues, lw_t_cache = refsFromLyricsWikia(missingdf[missingdf.song_key == "2001-96"])   
print
print "how many lyrics.wikia issues? ", len(lw_t_issues)
print "how many lyrics.wikia refs were downloaded? ", len(lw_t_refs)
print "how big is the lyrics.wikia cache? ", len(lw_t_cache)

print lw_t_cache

... song_key: 2001-96, artist: 3 Doors Down, song: Be Like That
('... attempting retrieval: ', u'http://lyrics.wikia.com/api.php?action=lyrics&fmt=json&artist=3+Doors+Down&song=Be+Like+That')

how many lyrics.wikia issues?  0
how many lyrics.wikia refs were downloaded?  1
how big is the lyrics.wikia cache?  1
{'2001-96': u'song = {\n\'artist\':\'3 Doors Down\',\n\'song\':\'Be Like That\',\n\'lyrics\':\'He spends his nights in California\\nWatching the stars on the big screen\\nAnd then he lies awake and he wonders\\n"Why can\\\'t that be me?"\\n\\n\\\'Cause in his life he\\\'s filled with all these good intentions\\nHe\\\'s left a lo[...]\',\n\'url\':\'http://lyrics.wikia.com/3_Doors_Down:Be_Like_That\'\n}\n'}


##Read Back Stored Files For Re-Processing 

In [177]:
# read lw_cache if exists
lw_cache = {}
try:
    with open('../../data/harvested/lw_cache.json') as fp:
        lw_cache = json.load(fp)
except Exception:
    print "... no lw_cache to read"

In [178]:
print "how big is current lw_cache? ", len(lw_cache)

how big is current lw_cache?  863


##Cleanup Lyrics.Wikia Metadata

In [217]:
def cleanLyricsWikiaMetaToJson(k,v,debug=False):    
    d = {}
    d['song_key'] = k
    d['meta'] = v
        
    if debug:
        print "key --> ", k    
        print "raw value -->\n", v
    
    qtoken = '":"'
    atoken = "':'"    
    q_mode = False
        
    qc = v.count(qtoken)
    ac = v.count(atoken)
    
    lyrics = ""
    llast = -1
    url = ""
    ulast = -1
    
    elyrics = False
    eurl = False
    
    # want lyrics and url
    #quot mode
    if qc > ac:    
        q_mode = True
        
        #lyrics
        try:
            lyrics = v.split('"lyrics{}'.format(qtoken),1)[1]
            lyrics = lyrics[:lyrics.find('"url{}'.format(qtoken))]
            llast = lyrics.rfind('"')    
        except Exception as e:
            print "{} lyrics not parsed --> {}".format(k,e)
            elyrics = True
        
        #url
        try:
            url = v.split('"url{}'.format(qtoken),1)[1]
            ulast = url.rfind('"') 
        except Exception as e:
            print "{} url not parsed --> {}".format(k,e)    
            eurl = True
        
    #apos mode    
    else:
        q_mode = False
        
        #lyrics
        try:
            lyrics = v.split("'lyrics{}".format(atoken),1)[1]
            lyrics = lyrics[:lyrics.find("'url{}".format(atoken))]
            llast = lyrics.rfind("'")
        except Exception as e:
            print "{} lyrics not parsed --> {}".format(k,e)
            elyrics = True
        
        #url
        try:
            url = v.split("'url{}".format(atoken),1)[1]
            ulast = url.rfind("'")
        except Exception as e:
            print "{} url not parsed --> {}".format(k,e)    
            eurl = True
        
    if debug:
        print
        print "q_mode? {}, qc: {}, ac: {}".format(q_mode, qc, ac)
    
    # final parse on lyrics
    if lyrics:
        lyrics = lyrics[:llast]    
        # check for any lingering quotes within lyrics
        if lyrics.count('"') > 0:
            #only mess with them if they are not already escaped
            if lyrics.count('\\"') < 1:
                if debug:
                    print "...messing with lingering quotes in lyrics"
                lyrics = lyrics.replace('"','\\"')
            
    # final parse on url    
    if url:
        url = url[:ulast]
    
    # add to dictionary, even if empty.
    d['lyrics'] = lyrics
    d['url'] = url
    
    if debug:
        print
        print "lyrics -->\n", lyrics
        print
        print "url --> ", url
    
    return d, elyrics, eurl    

##Save as skeleton files
This is an ad-hoc task based on the status of processing, which is manually iterative to troubleshoot failed lyrics.

In [220]:
lw_c_issues = {}

In [222]:
"""
for k,v in lw_cache.iteritems():
    with open("../../data/harvested/lw-json/{}.json".format(k), 'w') as fp:
        ctuple = cleanLyricsWikiaMetaToJson(k,v)
        if ctuple[1] or ctuple[2]:
            lw_c_issues[k] = json.dumps(ctuple[0])
        
        # go ahead and save to file
        json.dump(ctuple[0], fp)
"""

1973-90 lyrics not parsed --> list index out of range


In [223]:
print "How many clean issues? ", len(lw_c_issues)

How many clean issues?  1


##If processing full run from here (or some version)

In [38]:
print "execution start --> {}".format(time.strftime('%a, %d %b %Y %H:%M:%S', time.localtime()))

execution start --> Sun, 22 Nov 2015 03:44:32


In [ ]:
%%time
# run full
# lw_refs, lw_issues, lw_cache = refsFromLyricsWikia(missingdf,seed_cache=lw_cache)

In [40]:
print "how many lyrics.wikia issues? ", len(lw_issues)
print "how many lyrics.wikia refs were downloaded? ", len(lw_refs)
print "how big is the lyrics.wikia cache? ", len(lw_cache)

how many lyrics.wikia issues?  0
how many lyrics.wikia refs were downloaded?  863
how big is the lyrics.wikia cache?  863


##Save Progress

In [82]:
# save lw_cache
with open('../../data/harvested/lw_cache.json', 'w') as fp:
    json.dump(lw_cache, fp)